In [9]:
import torchvision

caltech101 = torchvision.datasets.Caltech101(root="datasets/caltech101", download=True)
cifar10 = torchvision.datasets.CIFAR10(root="datasets/cifar10", download=True)

In [10]:
# Print the number of samples in each dataset
print("Number of samples in Caltech101: ", len(caltech101))
print("Number of samples in CIFAR10: ", len(cifar10))

# Print labels
print("Caltech101 labels: ", caltech101.categories)
print("CIFAR10 labels: ", cifar10.classes)

Number of samples in Caltech101:  8677
Number of samples in CIFAR10:  50000
Caltech101 labels:  ['Faces', 'Faces_easy', 'Leopards', 'Motorbikes', 'accordion', 'airplanes', 'anchor', 'ant', 'barrel', 'bass', 'beaver', 'binocular', 'bonsai', 'brain', 'brontosaurus', 'buddha', 'butterfly', 'camera', 'cannon', 'car_side', 'ceiling_fan', 'cellphone', 'chair', 'chandelier', 'cougar_body', 'cougar_face', 'crab', 'crayfish', 'crocodile', 'crocodile_head', 'cup', 'dalmatian', 'dollar_bill', 'dolphin', 'dragonfly', 'electric_guitar', 'elephant', 'emu', 'euphonium', 'ewer', 'ferry', 'flamingo', 'flamingo_head', 'garfield', 'gerenuk', 'gramophone', 'grand_piano', 'hawksbill', 'headphone', 'hedgehog', 'helicopter', 'ibis', 'inline_skate', 'joshua_tree', 'kangaroo', 'ketch', 'lamp', 'laptop', 'llama', 'lobster', 'lotus', 'mandolin', 'mayfly', 'menorah', 'metronome', 'minaret', 'nautilus', 'octopus', 'okapi', 'pagoda', 'panda', 'pigeon', 'pizza', 'platypus', 'pyramid', 'revolver', 'rhino', 'rooster',